## 코로나 국가별 확진자 수 추이 분석/시각화
- https://public.flourish.studio/visualisation/1887877/

### 날짜별 확진자수 최종 데이터프레임 읽기 (df_confirmed)

In [251]:
import pandas as pd
df_confirmed = pd.read_csv("COVID-19-master/final_df.csv")
df_confirmed.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/01/2020,4/02/2020,4/03/2020,4/04/2020,4/05/2020
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,110,110,120,170,174,237,273,281,299,349
1,Albania,0,0,0,0,0,0,0,0,0,...,186,197,212,223,243,259,277,304,333,361
2,Algeria,0,0,0,0,0,0,0,0,0,...,409,454,511,584,716,847,986,1171,1251,1320
3,Andorra,0,0,0,0,0,0,0,0,0,...,267,308,334,370,376,390,428,439,466,501
4,Angola,0,0,0,0,0,0,0,0,0,...,4,5,7,7,7,8,8,8,10,14


In [252]:
df_confirmed.shape

(182, 76)

### 국가명과 iso2 매칭 테이블 읽기 (country_info)
  - iso2 컬럼값으로 https://www.countryflags.io/ 에서 제공하는 국기 이미지 링크를 얻을 수 있음
  - 다음 데이터는 결측치로 변환됨
    - ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’
  - Namibia 국가의 iso2 값이 NA 이므로 결측치 변환을 막기 위해 다음과 같은 옵션 설정
    - keep_default_na=False : 디폴트 결측치 변환 데이터를 사용하지 않고, na_values로 지정한 데이터만 결측치로 변환
    - na_values='' : 결측치로 변환할 값을 지정

In [254]:
country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='')
country_info.head()

,Unnamed: 0,Unnamed: 0.1,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key
0,0,0,NaN,BW,NaN,NaN,NaN,NaN,NaN,Botswana,NaN,NaN,Botswana
1,1,1,NaN,BI,NaN,NaN,NaN,NaN,NaN,Burundi,NaN,NaN,Burundi
2,2,2,NaN,SL,NaN,NaN,NaN,NaN,NaN,Sierra Leone,NaN,NaN,Sierra Leone
3,3,3,4.0,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan
4,4,4,8.0,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania


In [255]:
country_info[country_info['Country_Region'] == 'Namibia']

,Unnamed: 0,Unnamed: 0.1,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key
115,115,115,516.0,NA,NAM,516.0,NaN,NaN,NaN,Namibia,-22.9576,18.4904,Namibia


In [256]:
country_info = country_info[['iso2', 'Country_Region']]
country_info.head()

,iso2,Country_Region
0,BW,Botswana
1,BI,Burundi
2,SL,Sierra Leone
3,AF,Afghanistan
4,AL,Albania


- 중복 행 제거

In [257]:
country_info.shape

(3560, 2)

In [258]:
country_info = country_info.drop_duplicates(subset='Country_Region', keep='last')
country_info.shape

(180, 2)

### 날짜별 국가별 확진자수와 국가별 iso2 값 병합

In [259]:
doc_final_country = pd.merge(df_confirmed, country_info, how='left', on='Country_Region')
doc_final_country.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/01/2020,4/02/2020,4/03/2020,4/04/2020,4/05/2020,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,110,120,170,174,237,273,281,299,349,AF
1,Albania,0,0,0,0,0,0,0,0,0,...,197,212,223,243,259,277,304,333,361,AL
2,Algeria,0,0,0,0,0,0,0,0,0,...,454,511,584,716,847,986,1171,1251,1320,DZ
3,Andorra,0,0,0,0,0,0,0,0,0,...,308,334,370,376,390,428,439,466,501,AD
4,Angola,0,0,0,0,0,0,0,0,0,...,5,7,7,7,8,8,8,10,14,AO


In [260]:
doc_final_country.shape

(182, 77)

#### 없는 데이터(NaN) 확인하기

In [261]:
doc_final_country.isnull().sum()

Country_Region    0
1/22/2020         0
1/23/2020         0
1/24/2020         0
1/25/2020         0
                 ..
4/02/2020         0
4/03/2020         0
4/04/2020         0
4/05/2020         0
iso2              4
Length: 77, dtype: int64

### 특정 컬럼에 없는 데이터(NaN)가 있는 경우, 해당 행들만 보여주기
- 특정 컬럼 조건에 맞는 행들만 추출 가능
  - dataframe[dataframe[컬럼명] 조건]

In [262]:
doc_final_country[doc_final_country['iso2'].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/01/2020,4/02/2020,4/03/2020,4/04/2020,4/05/2020,iso2
43,Cruise Ship,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
48,Diamond Princess,0,0,0,0,0,0,0,0,0,...,712,712,712,712,712,712,712,712,712,NaN
103,MS Zaandam,0,0,0,0,0,0,0,0,0,...,2,2,2,2,9,9,9,9,9,NaN
128,Others,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


#### 참고: 복합 조건은 
- dataframe[(조건1) & (조건2)] : 조건1과 조건2 모두 만족 (and 조건)
- dataframe[(조건1) | (조건2)] : 조건1 또는 조건2 만족 (or 조건)
- 조건에 괄호를 넣는 것이 오동작을 방지하기 위해 좋음

```
nan_rows = doc_final_country[(doc_final_country['iso2'].isnull()) & (doc_final_country['4/01/2020'] == 0)]
nan_rows.head()
```

### 특정 컬럼에 없는 데이터 삭제하기

In [263]:
doc_final_country = doc_final_country.dropna(subset=['iso2'])

In [264]:
doc_final_country[doc_final_country['iso2'].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/01/2020,4/02/2020,4/03/2020,4/04/2020,4/05/2020,iso2


### 국기 링크를 기존 컬럼 기반해서 만들어, 데이터프레임에 붙이기
- 필요 데이터
  - 국가명, 국기, 날짜별 확진자 수
- www.countryflags.io
  - AD 는 https://www.countryflags.io/AD/flat/64.png

- 국기 링크 만들기

In [265]:
iso2= 'AD'
flag_link = 'https://www.countryflags.io/' + iso2 + '/flat/64.png'
flag_link

'https://www.countryflags.io/AD/flat/64.png'

#### 참고: 데이터프레임에 신규 컬럼 추가하기
- 데이터 프레임에 신규 컬럼 추가하기
  - 데이터프레임[신규컬럼] = pd.Series(data=데이터)
    - 데이터는 행에 맞게, [0, 1, 2] 와 같은 형태로 넣을 수 있음
- 기존 데이터프레임 컬럼 값을 기반으로 신규 컬럼 추가하기
  - apply() 함수를 사용
  - 다음 create_flag_link 함수 적용 전에, 아래의 함수로 어떤 값이 인자로 들어오는지 확인해보기 

    ```
    def create_flag_link(row):
        flag_link = 'https://www.countryflags.io/' + row + '/flat/64.png'
        return flag_link
        
    doc_final_country['Country_Flag'] = doc_final_country['iso2'].apply(create_flag_link)
    ```

In [266]:
def create_flag_link(row):
    flag_link = 'https://www.countryflags.io/' + row + '/flat/64.png'
    return flag_link

doc_final_country['iso2'] = doc_final_country['iso2'].apply(create_flag_link)

In [267]:
doc_final_country.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/01/2020,4/02/2020,4/03/2020,4/04/2020,4/05/2020,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,110,120,170,174,237,273,281,299,349,https://www.countryflags.io/AF/flat/64.png
1,Albania,0,0,0,0,0,0,0,0,0,...,197,212,223,243,259,277,304,333,361,https://www.countryflags.io/AL/flat/64.png
2,Algeria,0,0,0,0,0,0,0,0,0,...,454,511,584,716,847,986,1171,1251,1320,https://www.countryflags.io/DZ/flat/64.png
3,Andorra,0,0,0,0,0,0,0,0,0,...,308,334,370,376,390,428,439,466,501,https://www.countryflags.io/AD/flat/64.png
4,Angola,0,0,0,0,0,0,0,0,0,...,5,7,7,7,8,8,8,10,14,https://www.countryflags.io/AO/flat/64.png


### 데이터 포멧 확인하기
<img src="https://www.fun-coding.org/00_Images/covid_ex_data_format.jpg" />

### 데이터프레임 컬럼 조정하기

#### 1. 데이터프레임에서 필요한 컬럼만 선택하기
- 데이터프레임.columns.tolist() : 컬럼명을 리스트로 변환

In [268]:
cols = doc_final_country.columns.tolist()
cols

['Country_Region',
 '1/22/2020',
 '1/23/2020',
 '1/24/2020',
 '1/25/2020',
 '1/26/2020',
 '1/27/2020',
 '1/28/2020',
 '1/29/2020',
 '1/30/2020',
 '1/31/2020',
 '2/01/2020',
 '2/02/2020',
 '2/03/2020',
 '2/04/2020',
 '2/05/2020',
 '2/06/2020',
 '2/07/2020',
 '2/08/2020',
 '2/09/2020',
 '2/10/2020',
 '2/11/2020',
 '2/12/2020',
 '2/13/2020',
 '2/14/2020',
 '2/15/2020',
 '2/16/2020',
 '2/17/2020',
 '2/18/2020',
 '2/19/2020',
 '2/20/2020',
 '2/21/2020',
 '2/22/2020',
 '2/23/2020',
 '2/24/2020',
 '2/25/2020',
 '2/26/2020',
 '2/27/2020',
 '2/28/2020',
 '2/29/2020',
 '3/01/2020',
 '3/02/2020',
 '3/03/2020',
 '3/04/2020',
 '3/05/2020',
 '3/06/2020',
 '3/07/2020',
 '3/08/2020',
 '3/09/2020',
 '3/10/2020',
 '3/11/2020',
 '3/12/2020',
 '3/13/2020',
 '3/14/2020',
 '3/15/2020',
 '3/16/2020',
 '3/17/2020',
 '3/18/2020',
 '3/19/2020',
 '3/20/2020',
 '3/21/2020',
 '3/22/2020',
 '3/23/2020',
 '3/24/2020',
 '3/25/2020',
 '3/26/2020',
 '3/27/2020',
 '3/28/2020',
 '3/29/2020',
 '3/30/2020',
 '3/31/2020',
 

#### 데이터프레임 컬럼 위치 변경

In [269]:
cols.remove('iso2')
cols.insert(1, 'iso2')

In [270]:
doc_final_country = doc_final_country[cols]

In [271]:
doc_final_country.head()

,Country_Region,iso2,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,...,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/01/2020,4/02/2020,4/03/2020,4/04/2020,4/05/2020
0,Afghanistan,https://www.countryflags.io/AF/flat/64.png,0,0,0,0,0,0,0,0,...,110,110,120,170,174,237,273,281,299,349
1,Albania,https://www.countryflags.io/AL/flat/64.png,0,0,0,0,0,0,0,0,...,186,197,212,223,243,259,277,304,333,361
2,Algeria,https://www.countryflags.io/DZ/flat/64.png,0,0,0,0,0,0,0,0,...,409,454,511,584,716,847,986,1171,1251,1320
3,Andorra,https://www.countryflags.io/AD/flat/64.png,0,0,0,0,0,0,0,0,...,267,308,334,370,376,390,428,439,466,501
4,Angola,https://www.countryflags.io/AO/flat/64.png,0,0,0,0,0,0,0,0,...,4,5,7,7,7,8,8,8,10,14


#### 컬럼명 변경

In [272]:
cols[1] = 'Country_Flag'

In [273]:
doc_final_country.columns = cols
doc_final_country.head()

,Country_Region,Country_Flag,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,...,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/01/2020,4/02/2020,4/03/2020,4/04/2020,4/05/2020
0,Afghanistan,https://www.countryflags.io/AF/flat/64.png,0,0,0,0,0,0,0,0,...,110,110,120,170,174,237,273,281,299,349
1,Albania,https://www.countryflags.io/AL/flat/64.png,0,0,0,0,0,0,0,0,...,186,197,212,223,243,259,277,304,333,361
2,Algeria,https://www.countryflags.io/DZ/flat/64.png,0,0,0,0,0,0,0,0,...,409,454,511,584,716,847,986,1171,1251,1320
3,Andorra,https://www.countryflags.io/AD/flat/64.png,0,0,0,0,0,0,0,0,...,267,308,334,370,376,390,428,439,466,501
4,Angola,https://www.countryflags.io/AO/flat/64.png,0,0,0,0,0,0,0,0,...,4,5,7,7,7,8,8,8,10,14


### 최종 가공 완료 파일 저장

In [274]:
doc_final_country.to_csv("COVID-19-master/final_covid_data_for_graph.csv")

### 그래프 만들기
- https://app.flourish.studio/ 로그인
- New visualization 선택 -> Bar Chart Race 선택 -> 데이터 수정 (파일 업로드)